In [ ]:
!pip3 install pandas
!pip3 install numpy

Importaciones

In [ ]:
import pandas as pd
import numpy as np

Carga de los datos y filtrado

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

Análisis de los datos

In [66]:
# Número de instancias y características
print(f'Número de instancias: {wind_ava.shape[0]}')
print(f'Número de características: {wind_ava.shape[1]}')

# Variables categóricas y numéricas
categorical_vars = wind_ava.select_dtypes(include=['object']).columns
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns
print(f'Variables categóricas: {categorical_vars}')
print(f'Variables numéricas: {numerical_vars}')

# Valores faltantes
zero_count = (wind_ava == 0).sum().sum()
print(f"Número de valores que son 0: {zero_count}")

# Columnas constantes
constant_columns = [col for col in wind_ava.columns if wind_ava[col].nunique() <= 1]
print(f'Columnas constantes: {constant_columns}')

# Determinar si es un problema de regresión o clasificación
if wind_ava['energy'].nunique() > 2:
    print('Es un problema de regresión')
else:
    print('Es un problema de clasificación')

Número de instancias: 4748
Número de características: 24
Variables categóricas: Index(['datetime'], dtype='object')
Variables numéricas: Index(['energy', 'p54.162.13', 'p55.162.13', 'cape.13', 'p59.162.13',
       'lai_lv.13', 'lai_hv.13', 'u10n.13', 'v10n.13', 'sp.13', 'stl1.13',
       'u10.13', 'v10.13', 't2m.13', 'stl2.13', 'stl3.13', 'iews.13',
       'inss.13', 'stl4.13', 'fsr.13', 'flsr.13', 'u100.13', 'v100.13'],
      dtype='object')
Número de valores que son 0: 1492
Columnas constantes: []
Es un problema de regresión


In [67]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=0.3, random_state=100472166)

# Definir los métodos de escalado para evaluar
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

# Crear un imputador para reemplazar los valores 0 con la media
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Evaluar KNN con diferentes métodos de escalado
results = {}
for scaler_name, scaler in scalers.items():
    # Imputar los valores 0 en los datos de entrenamiento
    X_train_imputed = imputer.fit_transform(X_train)
    
    # Escalar los datos de entrenamiento
    X_train_scaled = scaler.fit_transform(X_train_imputed)
    
    # Entrenar el modelo KNN
    knn = KNeighborsRegressor()
    knn.fit(X_train_scaled, y_train)
    
    # Imputar los valores 0 en los datos de prueba
    X_test_imputed = imputer.transform(X_test)
    
    # Escalar los datos de prueba
    X_test_scaled = scaler.transform(X_test_imputed)
    
    # Evaluar el modelo
    score = knn.score(X_test_scaled, y_test)
    
    # Almacenar los resultados
    results[scaler_name] = score

for scaler_name, score in results.items():
    print(f"Variable Name: {scaler_name}")
    print(f"Variable Score: {score}")
    print()

# Elegir el método de escalado con el mejor rendimiento
mejor_scaler = max(results, key=results.get)
print(f"El mejor método de escalado es {mejor_scaler} con un puntaje de {results[mejor_scaler]}")



Variable Name: StandardScaler
Variable Score: 0.8833083572560653

Variable Name: MinMaxScaler
Variable Score: 0.9256423746546902

Variable Name: RobustScaler
Variable Score: 0.8419084199545743

El mejor método de escalado es MinMaxScaler con un puntaje de 0.9256423746546902
